<a href="https://colab.research.google.com/github/Saauirr/KalmanPyQt/blob/master/%D0%A2%D0%9E%D0%9E%20ALAXAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Backend (Python Flask)

In [ ]:
# app.py
from flask import Flask, request, jsonify
import sqlite3

app = Flask(__name__)

def init_db():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS items
                 (id INTEGER PRIMARY KEY, name TEXT, value REAL)''')
    conn.commit()
    conn.close()

@app.route('/items', methods=['GET'])
def get_items():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()
    c.execute('SELECT * FROM items')
    items = c.fetchall()
    conn.close()
    return jsonify(items)

@app.route('/items', methods=['POST'])
def add_item():
    new_item = request.get_json()
    name = new_item['name']
    value = new_item['value']
    conn = sqlite3.connect('database.db')
    c = conn.cursor()
    c.execute('INSERT INTO items (name, value) VALUES (?, ?)', (name, value))
    conn.commit()
    conn.close()
    return jsonify({"status": "success"}), 201

@app.route('/items/<int:item_id>', methods=['PUT'])
def update_item(item_id):
    updated_item = request.get_json()
    name = updated_item['name']
    value = updated_item['value']
    conn = sqlite3.connect('database.db')
    c = conn.cursor()
    c.execute('UPDATE items SET name = ?, value = ? WHERE id = ?', (name, value, item_id))
    conn.commit()
    conn.close()
    return jsonify({"status": "success"})

@app.route('/items/<int:item_id>', methods=['DELETE'])
def delete_item(item_id):
    conn = sqlite3.connect('database.db')
    c = conn.cursor()
    c.execute('DELETE FROM items WHERE id = ?', (item_id,))
    conn.commit()
    conn.close()
    return jsonify({"status": "success"})

if __name__ == '__main__':
    init_db()
    app.run(debug=True)


Frontend (Flutter)

In [ ]:
// main.dart
import 'package:flutter/material.dart';
import 'dart:convert';
import 'package:http/http.dart' as http;

void main() => runApp(MyApp());

class MyApp extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Flutter + Flask Demo',
      theme: ThemeData(
        primarySwatch: Colors.blue,
      ),
      home: ItemListPage(),
    );
  }
}

class ItemListPage extends StatefulWidget {
  @override
  _ItemListPageState createState() => _ItemListPageState();
}

class _ItemListPageState extends State<ItemListPage> {
  List items = [];

  @override
  void initState() {
    super.initState();
    fetchItems();
  }

  fetchItems() async {
    final response = await http.get(Uri.parse('http://127.0.0.1:5000/items'));
    if (response.statusCode == 200) {
      setState(() {
        items = json.decode(response.body);
      });
    } else {
      throw Exception('Failed to load items');
    }
  }

  addItem(String name, double value) async {
    final response = await http.post(
      Uri.parse('http://127.0.0.1:5000/items'),
      headers: <String, String>{
        'Content-Type': 'application/json; charset=UTF-8',
      },
      body: jsonEncode(<String, dynamic>{
        'name': name,
        'value': value,
      }),
    );
    if (response.statusCode == 201) {
      fetchItems();
    } else {
      throw Exception('Failed to add item');
    }
  }

  updateItem(int id, String name, double value) async {
    final response = await http.put(
      Uri.parse('http://127.0.0.1:5000/items/$id'),
      headers: <String, String>{
        'Content-Type': 'application/json; charset=UTF-8',
      },
      body: jsonEncode(<String, dynamic>{
        'name': name,
        'value': value,
      }),
    );
    if (response.statusCode == 200) {
      fetchItems();
    } else {
      throw Exception('Failed to update item');
    }
  }

  deleteItem(int id) async {
    final response = await http.delete(
      Uri.parse('http://127.0.0.1:5000/items/$id'),
    );
    if (response.statusCode == 200) {
      fetchItems();
    } else {
      throw Exception('Failed to delete item');
    }
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text('Item List'),
      ),
      body: ListView.builder(
        itemCount: items.length,
        itemBuilder: (context, index) {
          final item = items[index];
          return ListTile(
            title: Text(item[1]),
            subtitle: Text(item[2].toString()),
            trailing: Row(
              mainAxisSize: MainAxisSize.min,
              children: <Widget>[
                IconButton(
                  icon: Icon(Icons.edit),
                  onPressed: () {
                    // Update item code here
                  },
                ),
                IconButton(
                  icon: Icon(Icons.delete),
                  onPressed: () {
                    deleteItem(item[0]);
                  },
                ),
              ],
            ),
          );
        },
      ),
      floatingActionButton: FloatingActionButton(
        onPressed: () {
          // Add item code here
        },
        child: Icon(Icons.add),
      ),
    );
  }
}


Объяснение

    Backend (Flask):
        Инициализация базы данных SQLite.
        Реализация CRUD операций (создание, чтение, обновление, удаление) через HTTP API.

    Frontend (Flutter):
        Приложение на Flutter с запросами к Flask API для выполнения CRUD операций.
        Интерфейс для отображения списка элементов и выполнения действий (добавление, редактирование, удаление).